<a href="https://colab.research.google.com/github/ricedavis/CSCE580-Fall2025/blob/main/ProjectB/steps_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset

In [7]:
# import dataset
from google.colab import files
uploaded = files.upload()  # then select your CSV file


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [8]:
import pandas as pd

df = pd.read_csv("IMDB Dataset.csv")
print(df.head())
print(df['sentiment'].value_counts())  # see how many positive/negative reviews


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


# Preprocessing

In [9]:
import re

# minor text cleaning for DistilBERT
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)  # remove punctuation/numbers
    return text

df['clean_review'] = df['review'].apply(clean_text)


In [10]:
# convert sentiment to 0/1
df['label'] = df['sentiment'].map({'negative': 0, 'positive': 1})

For any supervised ML task, it's important to divide your dataset into seperate subsets for training, validation and testing. This ensures that the model learns patterns from one subset and is evaluated on unseen data.

In [11]:
from sklearn.model_selection import train_test_split

# using sklearn to split data

X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)

In [12]:
!pip install transformers

In [13]:
# tokenizing for DistilBERT

from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

MAX_LENGTH = 128

def encode_texts(texts):
    return tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

train_encodings = encode_texts(X_train)
valid_encodings = encode_texts(X_valid)
test_encodings = encode_texts(X_test)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [23]:
# Wrap encodings into PyTorch datasets

import torch
from torch.utils.data import Dataset, DataLoader

class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item


train_dataset = IMDBDataset(train_encodings, y_train)
valid_dataset = IMDBDataset(valid_encodings, y_valid)
test_dataset = IMDBDataset(test_encodings, y_test)


In [26]:
# Create DataLoaders

BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


# Fine-Tuning DistilBERT

In [16]:
from torch import nn, optim
from transformers import DistilBertForSequenceClassification
from tqdm.auto import tqdm
from torch.optim import AdamW

In [17]:
# Set device

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using device:", device)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

Using device: cuda


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [18]:
# optimizer and loss

optimizer = AdamW(model.parameters(), lr=5e-5)  # common starting learning rate
loss_fn = nn.CrossEntropyLoss()


In [19]:
EPOCHS = 3

for epoch in range(EPOCHS):
    # ---------- Training ----------
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)

    # ---------- Validation ----------
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            valid_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_valid_loss = valid_loss / len(valid_loader)
    valid_acc = correct / total

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_valid_loss:.4f} | Val Acc: {valid_acc:.4f}")



Training Epoch 1:   0%|          | 0/2250 [00:00<?, ?it/s]

/tmp/ipython-input-30319435.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1 | Train Loss: 0.3424 | Val Loss: 0.3034 | Val Acc: 0.8755


Training Epoch 2:   0%|          | 0/2250 [00:00<?, ?it/s]

Epoch 2 | Train Loss: 0.2082 | Val Loss: 0.2854 | Val Acc: 0.8785


Training Epoch 3:   0%|          | 0/2250 [00:00<?, ?it/s]

Epoch 3 | Train Loss: 0.1048 | Val Loss: 0.3308 | Val Acc: 0.8690


Test evaluation function:

In [24]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [27]:
fine_tuned_acc = evaluate(model, test_loader)
print("Fine-tuned DistilBERT Accuracy:", fine_tuned_acc)


/tmp/ipython-input-3808085645.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Fine-tuned DistilBERT Accuracy: 0.8775


# Base Model Comparison

In [28]:
# Load the base DistilBERT model (NOT fine-tuned)
base_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased'
).to(device)

base_acc = evaluate(base_model, test_loader)
print("Base DistilBERT Accuracy:", base_acc)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3808085645.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Base DistilBERT Accuracy: 0.5504


Accuracy: 0.5504

In [29]:
# Fine tuned DistilBERT
fine_tuned_acc = evaluate(model, test_loader)
print("Fine-tuned DistilBERT Accuracy:", fine_tuned_acc)


/tmp/ipython-input-3808085645.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Fine-tuned DistilBERT Accuracy: 0.8775


Accuracy: 0.8775

GPT comparison:

In [33]:
df_test = pd.DataFrame({
    "review": X_test,
    "label": y_test
})

In [30]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn.functional as F

gpt_tok = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2").to(device)
gpt_tok.pad_token = gpt_tok.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [31]:
# Define scoring function
def gpt2_sentiment(text):
    # Two prompts: one predicting positive, one negative
    prompt_pos = f"{text}\nSentiment: positive"
    prompt_neg = f"{text}\nSentiment: negative"

    def score(prompt):
        inputs = gpt_tok(prompt, return_tensors='pt', truncation=True, max_length=256).to(device)
        with torch.no_grad():
            out = gpt_model(**inputs, labels=inputs["input_ids"])
        return -out.loss.item()  # higher = more likely

    s_pos = score(prompt_pos)
    s_neg = score(prompt_neg)

    return 1 if s_pos > s_neg else 0


In [34]:
# Evaluate GPT2 on small sample
sample_df = df_test.sample(200, random_state=42)

correct = 0
for review, label in zip(sample_df['review'], sample_df['label']):
    pred = gpt2_sentiment(review)
    correct += (pred == label)

gpt2_acc = correct / len(sample_df)
print("GPT-2 Accuracy (sample of 200):", gpt2_acc)


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


GPT-2 Accuracy (sample of 200): 0.635


# Classical Machine Learning Model:

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1,2),
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [36]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=300)
clf.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=300)

In [37]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test_tfidf)
logreg_acc = accuracy_score(y_test, y_pred)

print("TF-IDF + Logistic Regression Accuracy:", logreg_acc)


TF-IDF + Logistic Regression Accuracy: 0.8998


Final Accuracies:


*   Base DistilBERT: 0.5504
*   Fine-tuned DistilBERT: 0.8775
*   GPT2 (zero shot): 0.635
*   Classical ML (logistic regression): 0.8998



